In [5]:
class Card() :
    def __init__(self, value, color) :
        if value.lower() in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king'] :
            self.value = value.title()
        else :
            raise 'ValueError'
        if color.lower() in ['heart', 'spade', 'diamond', 'club'] :
            self.color = color.title()
        else :
            raise 'ValueError'    
    
class Player() :
    def __init__(self, name) :
        self.name = name
        self.hand = []
        self.isclone = False
        self.bet = 0
        self.score = 0
        self.stack = 500

    def empty_hand(self) :
        self.hand = []

    def draw_card(self, deck) :
        self.hand.append(deck[-1])
        deck.pop()

In [ ]:
import random

def shuffled_deck() :
    deck = [Card(value, color) for value in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king'] for color in ['heart', 'spade', 'diamond', 'club']]
    random.shuffle(deck)
    return deck

In [6]:
def init_players() :
    num_players = '0'
    list_players = []
    while num_players not in ['1', '2', '3', '4'] :
        num_players = input("Please choose the number of players. Enter a number between 1 and 4.")
    for i in range(int(num_players)) :
        next_player = ''
        while next_player in ['', 'Dealer'] or next_player in list_players :
            next_player = input(f"Enter the name of player {i+1} other than \"Dealer\" or the name of another player.")
        list_players.append(next_player)
    return [Player(name) for name in list_players]

dealer = Player("Dealer")

In [7]:
def enter_bets(players) :
    deck = shuffled_deck()
    dealer.draw_card(deck)
    input(f"The dealer has drawn a {dealer.hand[0].value.title()} of {dealer.hand[0].color.title()}. Press enter to continue")
    active_players = players
    for player in players :
        bet = '1'
        while bet not in [str(item) for item in range(0, 101, 2)] or int(bet) > player.stack if bet.isdigit() else True :
            bet = input(f"Player {player.name}, your current stack is {player.stack}. Please enter your bet. Choose any even number between 0 and 100 not greater than your stack. Please consider that a bet of 0 will make you leave the game and make your stack of chips final.")
        player.bet = int(bet)
        if bet == '0' :
            active_players.remove(player)
    return (active_players, deck)

def hand_value (hand) :
    there_is_an_ace = False
    h_v = 0
    for card in hand :
        if card.value in [str(item) for item in range(1, 11)] :
            h_v += int(card.value)
            if card.value == '1' :
                there_is_an_ace = True
        else :
            h_v += 10
    if there_is_an_ace and h_v <= 11 :
        return h_v + 10
    else :
        return h_v

def string_from_hand (hand) :
    s = 'nothing'
    if hand != [] :
        s = f'a {hand[0].value} of {hand[0].color}'
    for card in hand[1:len(hand)] :
        s = s+f' and a {card.value} of {card.color}'
    return s

In [8]:
def clone(player):
    c = Player(player.name+', your second hand')
    c.isclone = True
    c.bet = player.bet
    return c
    
def split (index, players, deck) :
    active_players = []
    p = players[index]
    c = clone(p)
    for i in range(index-1) :
        active_players.append(players[i])
    cardclone = p.hand[1]
    c.hand = [cardclone]
    p.hand.pop()
    p.draw_card(deck)
    c.draw_card(deck)
    active_players.append(p)
    active_players.append(c)
    for i in range(index+1, len(players)) :
        active_players.append(players[i])
    return active_players



In [9]:
def round(players, deck) :
    if players == [] :
        return[]
    else :
        active_players = [player for player in players]
        index = 0
        
        for pl in players :
            pl.draw_card(deck); pl.draw_card(deck)
            if pl.hand[0].value == pl.hand[1].value and pl.stack >= pl.bet * 2 :
                do_pl_split = ''
                while do_pl_split.lower() not in ['yes', 'no'] :
                    do_pl_split = input(f"Player {pl.name}, you have a double as a starting hand. You have {string_from_hand(pl.hand)}. Do you want to split your hand and bet an extra {pl.bet} chips? Enter \"Yes\" or \"No\".")
                if do_pl_split.lower() == 'yes' :
                    active_players = split(index, active_players, deck)
                    index += 2
                else :
                    index += 1
        
        for pl in active_players :
            do_pl_hit = ''
            while hand_value(pl.hand) <= 21 and do_pl_hit.lower() != 'stand':
                while do_pl_hit.lower() not in ['hit', 'stand'] :
                    do_pl_hit = input(f'Player {pl.name}. You have {string_from_hand(pl.hand)} in your hand. Your score is {hand_value(pl.hand)}. Do you want to hit or stand? Please enter \"Hit\" if you want to hit and \"Stand\" if you want to stand')
                if do_pl_hit.lower() == 'hit' :
                    pl.draw_card(deck)
                    do_pl_hit = ''
            if hand_value(pl.hand) > 21 :
                input(f'Player {pl.name}. You drew {string_from_hand([pl.hand[-1]])}. Your score is {hand_value(pl.hand)}. You are busted! Press Enter to continue.')
            else :
                input(f'Player {pl.name}. Your final score for this round is {hand_value(pl.hand)}. Press Enter to continue.')
        
        while hand_value(dealer.hand) < 17 :
            dealer.draw_card(deck)
        score_dealer = hand_value(dealer.hand)
        input(f"The Dealer drew {string_from_hand(dealer.hand)}. The Dealer's final score for this round is {score_dealer}. Press Enter to continue.")
        if score_dealer > 21 :
            input("The Dealer is busted! Press Enter to continue.")
            score_dealer = 0

        for i in range(len(active_players)) :
            pl_i = active_players[i]
            score_i = hand_value(pl_i.hand)
            if not pl_i.isclone :
                if score_i > score_dealer and score_i < 21 :
                    pl_i.stack += pl_i.bet
                    input(f'Player {pl_i.name}. You won your bet! You earned {pl_i.bet} extra chips! Press Enter to continue.')
                elif score_i > score_dealer and score_i == 21 :
                    pl_i.stack += pl_i.bet * 3 // 2
                    input(f'Player {pl_i.name}. You won your bet with a blackjack! Amazing! You earned {pl_i.bet * 3 // 2} extra chips! Press Enter to continue.')
                else :
                    pl_i.stack -= pl_i.bet
                    input(f'Player {pl_i.name}. You lost your bet. You lost {pl_i.bet} chips. Press Enter to continue.')
            else :
                if score_i > score_dealer and score_i < 21 :
                    active_players[i-1].stack += pl_i.bet
                    input(f'Player {pl_i.name}. You won your bet! You earned {pl_i.bet} extra chips! Press Enter to continue.')
                elif score_i > score_dealer and score_i == 21 :
                    active_players[i-1].stack += pl_i.bet * 3 // 2
                    input(f'Player {pl_i.name}. You won your bet with a blackjack! Amazing! You earned {pl_i.bet * 3 // 2} extra chips! Press Enter to continue.')
                else :
                    active_players[i-1].stack -= pl_i.bet
                    input(f'Player {pl_i.name}. You lost your bet. You lost {pl_i.bet} chips. Press Enter to continue.')
        for player in active_players :
            player.empty_hand()
        dealer.empty_hand()
        return [player for player in active_players if not player.isclone]

In [10]:
def blackjack() :
    players = init_players()
    winner = [players[0]]
    active_players = [player for player in players]
    while active_players != [] :
        e_b = enter_bets(active_players)
        active_players = round(e_b[0], e_b[1])
    
    for player in players :
        input(f'Player {player.name}. Your final stack has {player.stack} chips. Well played! Press Enter to continue.')
        if player.stack > winner[0].stack :
            winner = [player]
        elif player.stack == winner[0].stack and player != winner[0] :
            winner.append(player)
    for player in winner :
        if player.stack > 500 :
            input(f'Player {player.name}. Well played! You won this game with a high score of {player.stack}!')

In [ ]:
blackjack()
